In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *

In [2]:
PATH="/home/kirana/Documents/phd"
DATAPATH="/home/kirana/Documents/phd/data/experiment/YTS"

## Dataset

In [3]:
[df_train,df_valid,itos, train_tokens, valid_tokens, trn_lm, val_lm]=pickle.load(open(f'{DATAPATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [4]:
type(df_train)

pandas.core.frame.DataFrame

In [5]:
df_train.head()

,text,label,dstype,words,tokens
333,Nice,0,train,"[ \n , xxbos, xxfld, 1, xxmaj, nice]","[5, 6, 7, 4, 2, 186]"
1092,"Hey, join me on tsū, a publishing platform whe...",1,train,"[ \n , xxbos, xxfld, 1, xxmaj, hey, ,, join, m...","[5, 6, 7, 4, 2, 82, 18, 403, 33, 23, 0, 18, 16..."
608,she is beautiful but it is not American!﻿,0,train,"[ \n , xxbos, xxfld, 1, she, is, beautiful, bu...","[5, 6, 7, 4, 85, 27, 246, 53, 25, 27, 86, 2, 0]"
672,"Like Gorlin-Goltz Syndrome Support Community, ...",1,train,"[ \n , xxbos, xxfld, 1, xxmaj, like, xxmaj, go...","[5, 6, 7, 4, 2, 35, 2, 0, 26, 2, 0, 2, 0, 2, 2..."
1286,I personally have never been in a abusive rela...,1,train,"[ \n , xxbos, xxfld, 1, i, personally, have, n...","[5, 6, 7, 4, 10, 0, 50, 269, 295, 36, 16, 0, 0..."


In [6]:
df=pd.concat([df_train,df_valid],axis=0)

In [7]:
df['dstype'].value_counts()

train    1363
test      391
valid     194
Name: dstype, dtype: int64

In [8]:
df_train=df.loc[df['dstype']!='test']
df_test=df.loc[df['dstype']=='test']

In [9]:
df_train.shape, df_test.shape

((1557, 5), (391, 5))

In [10]:
df_train.head()

,text,label,dstype,words,tokens
333,Nice,0,train,"[ \n , xxbos, xxfld, 1, xxmaj, nice]","[5, 6, 7, 4, 2, 186]"
1092,"Hey, join me on tsū, a publishing platform whe...",1,train,"[ \n , xxbos, xxfld, 1, xxmaj, hey, ,, join, m...","[5, 6, 7, 4, 2, 82, 18, 403, 33, 23, 0, 18, 16..."
608,she is beautiful but it is not American!﻿,0,train,"[ \n , xxbos, xxfld, 1, she, is, beautiful, bu...","[5, 6, 7, 4, 85, 27, 246, 53, 25, 27, 86, 2, 0]"
672,"Like Gorlin-Goltz Syndrome Support Community, ...",1,train,"[ \n , xxbos, xxfld, 1, xxmaj, like, xxmaj, go...","[5, 6, 7, 4, 2, 35, 2, 0, 26, 2, 0, 2, 0, 2, 2..."
1286,I personally have never been in a abusive rela...,1,train,"[ \n , xxbos, xxfld, 1, i, personally, have, n...","[5, 6, 7, 4, 10, 0, 50, 269, 295, 36, 16, 0, 0..."


In [11]:
type(df_train['tokens'])

pandas.core.series.Series

In [12]:
bs=52 # 52 - Jeremey, 20 - default

In [13]:
bptt= 70 #70 - Jeremey, 35 - default

In [14]:
df_train.head()

,text,label,dstype,words,tokens
333,Nice,0,train,"[ \n , xxbos, xxfld, 1, xxmaj, nice]","[5, 6, 7, 4, 2, 186]"
1092,"Hey, join me on tsū, a publishing platform whe...",1,train,"[ \n , xxbos, xxfld, 1, xxmaj, hey, ,, join, m...","[5, 6, 7, 4, 2, 82, 18, 403, 33, 23, 0, 18, 16..."
608,she is beautiful but it is not American!﻿,0,train,"[ \n , xxbos, xxfld, 1, she, is, beautiful, bu...","[5, 6, 7, 4, 85, 27, 246, 53, 25, 27, 86, 2, 0]"
672,"Like Gorlin-Goltz Syndrome Support Community, ...",1,train,"[ \n , xxbos, xxfld, 1, xxmaj, like, xxmaj, go...","[5, 6, 7, 4, 2, 35, 2, 0, 26, 2, 0, 2, 0, 2, 2..."
1286,I personally have never been in a abusive rela...,1,train,"[ \n , xxbos, xxfld, 1, i, personally, have, n...","[5, 6, 7, 4, 10, 0, 50, 269, 295, 36, 16, 0, 0..."


In [15]:
df_train['label'].value_counts()

1    788
0    769
Name: label, dtype: int64

In [16]:
df_valid['label'].value_counts()

0    111
1     84
Name: label, dtype: int64

In [17]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'xxup', '1']

In [18]:
padlen=1400
padding_idx=1

In [19]:
df_train['n_tok']=df_train['tokens'].apply(len)

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
df_train['n_tok'].describe()

count    1557.000000
mean       27.473988
std        32.272341
min         5.000000
25%        11.000000
50%        16.000000
75%        27.000000
max       287.000000
Name: n_tok, dtype: float64

In [21]:
df_train.sort_values(by='n_tok', inplace=True)

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
df_valid['n_tok']=df_valid['tokens'].apply(len)
df_valid.sort_values(by='n_tok', inplace=True)

In [23]:
df_test['n_tok']=df_test['tokens'].apply(len)
df_test.sort_values(by='n_tok', inplace=True)

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
for i in range(0,20):
    x=df_train['tokens'].values[i]
    y=df_train['label'].values[i]    
    out=np.ones(padlen)
    if len(x) < padlen:
        out[:len(x)]=x
    else:
        out=x[:padlen]
    print (out)
    print (y)
    print (len(out))
    if i>10:
        break

[5. 6. 7. 4. ... 1. 1. 1. 1.]
1
1400
[5. 6. 7. 4. ... 1. 1. 1. 1.]
0
1400
[5. 6. 7. 4. ... 1. 1. 1. 1.]
0
1400
[5. 6. 7. 4. ... 1. 1. 1. 1.]
0
1400
[5. 6. 7. 4. ... 1. 1. 1. 1.]
0
1400
[5. 6. 7. 4. ... 1. 1. 1. 1.]
0
1400
[5. 6. 7. 4. ... 1. 1. 1. 1.]
0
1400
[5. 6. 7. 4. ... 1. 1. 1. 1.]
0
1400
[5. 6. 7. 4. ... 1. 1. 1. 1.]
0
1400
[5. 6. 7. 4. ... 1. 1. 1. 1.]
0
1400
[5. 6. 7. 4. ... 1. 1. 1. 1.]
0
1400
[5. 6. 7. 4. ... 1. 1. 1. 1.]
0
1400


In [25]:
df_train.shape

(1557, 6)

In [26]:
df_train['label'].value_counts(), df_valid['label'].value_counts(),df_test['label'].value_counts()

(1    788
 0    769
 Name: label, dtype: int64, 0    111
 1     84
 Name: label, dtype: int64, 1    217
 0    174
 Name: label, dtype: int64)

In [27]:
df_train.shape, df_valid.shape

((1557, 6), (195, 6))

In [28]:
padlen

1400

In [29]:
class ds_sentiment (Dataset):
    def __init__ (self,df,bs,padlen=64,xvar='tokens',yvar='label',len_var='n_tok',padding_idx=1):
        self.x,self.y,self.padlen,self.padding_idx,self.len_var,self.bs=\
            df[xvar],df[yvar],padlen,padding_idx,df[len_var],bs
        self.len_var=self.len_var.clip(0,padlen)
    
    def pad (self,x):
        out=np.ones(self.padlen)*self.padding_idx
        out=out.astype(int)
        if len(x)>=self.padlen:
            out[:]=x[:self.padlen]
        else:
            out[:len(x)]=x
        return out
        
    def __getitem__(self,idx):
        #import pdb
        #pdb.set_trace()
        return self.pad(self.x.iloc[idx]),self.y.iloc[idx],self.len_var.iloc[idx]
    
    def __len__(self):
        return len(self.x)        


In [30]:
#padlen=max(df_train['n_tok'])
padlen=1400

In [31]:
padlen

1400

In [32]:
dstrain=ds_sentiment(df_train,bs,padlen)
dsvalid=ds_sentiment(df_valid,bs,padlen)
dstest=ds_sentiment(df_test,bs,padlen)

In [33]:
dltrain=DataLoader(dstrain,bs,True)
dlvalid=DataLoader(dsvalid,bs,False)
dltest=DataLoader(dstest,bs,False)

In [34]:
for xb,yb,xlen in dltrain:
    break

In [35]:
xb, yb, xlen

(tensor([[5, 6, 7,  ..., 1, 1, 1],
         [5, 6, 7,  ..., 1, 1, 1],
         [5, 6, 7,  ..., 1, 1, 1],
         ...,
         [5, 6, 7,  ..., 1, 1, 1],
         [5, 6, 7,  ..., 1, 1, 1],
         [5, 6, 7,  ..., 1, 1, 1]]),
 tensor([0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
         0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
         1, 0, 1, 0]),
 tensor([  8,  10,  13,  17,  11,  12,  13,  38,  16,  22,  13,  19,   9,   9,
          18,   9, 120,   6,  11,  29,  19,   9,  11,  76,  33,  66,  96,  84,
          17,   9,   6,  21,  86, 122,  13,  80,  23,  18,  18,  40,  23, 135,
          14,  46,  11,  15,   7,  13,   7,  22,  84,   8]))

## Model Architecture

In [36]:
from torch import nn
import torch
from torch.autograd import Variable

In [37]:
bs,bptt

(52, 70)

In [38]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=400#400
n_layers= 2 # 2
dropout=0.5 # 0.5
wd=1e-7
bidirectional=True
dropout_e=0.2 # 0.5 - changing to 0.4, 0.3 or any dropout value did not make much difference
dropout_o=0.5 #0.5
n_out=1

In [39]:
xb.shape,xlen.shape

(torch.Size([52, 1400]), torch.Size([52]))

In [40]:
df_train['label'].value_counts()

1    788
0    769
Name: label, dtype: int64

In [41]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [42]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'xxup', '1']

In [43]:
import torch

In [44]:
m = nn.AdaptiveAvgPool1d(5)
input = torch.randn(1, 64, 8)
input.size()

torch.Size([1, 64, 8])

In [45]:
m(input).size()

torch.Size([1, 64, 5])

In [46]:
class sentiment_classifier (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,n_out=1,padding_idx=1,n_filters=100,filter_sizes=[3,4,5]):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx,self.padding_idx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx,padding_idx
        self.n_out,self.n_filters,self.filter_sizes=n_out,n_filters,filter_sizes
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        
        self.create_architecture()
        if pretrain_mtx is not None:
            print (f'initializing glove with {pretrain_mtx.shape}')
            self.initialize_glove()
        self.init_hidden()
        self.criterion=nn.BCEWithLogitsLoss()
    
    def set_dropouts(self, dropout, dropout_o, dropout_e):
        self.dropout, self.dropout_o, self.dropout_e = dropout, dropout_o, dropout_e
    
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
         
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True

    def initialize_glove(self):
        self.encoder.weight.data.copy_(torch.Tensor(self.pretrain_mtx))
    
    def init_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    

    def create_architecture(self):
        ###################################
        # Embedding layer - common to both
        ###################################
        self.dropout_enc=nn.Dropout(self.dropout_e)
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
        
        #######################################
        # For RNN #############################
        #######################################
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
         # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=self.bidirectional)
          # embs are going to be of shape n_batch * n_seq * n_emb
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        self.avg_pool1d=torch.nn.AdaptiveAvgPool1d(1)
        self.max_pool1d=torch.nn.AdaptiveMaxPool1d(1)
        
    
        #######################################
        # For CNN #############################
        #######################################    
        #embedding dimension is the "depth" of the filter and the number of tokens in the sentence is the width.
        self.conv_0=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[0])
        self.conv_1=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[1])
        self.conv_2=torch.nn.Conv1d(self.n_emb,self.n_filters,kernel_size=self.filter_sizes[2])
        
        self.fc=nn.Linear(len(self.filter_sizes)*self.n_filters+self.n_hidden*4,self.n_out)
        

        
    def forward (self,Xb,Yb,Xb_lengths):
        
        ####RNN PORTION
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        packed_embs = pack_padded_sequence(embs,Xb_lengths,batch_first=True, enforce_sorted=False)
        lstm_out,(hidden,cell)=self.lstm(packed_embs)
        lstm_out,lengths=pad_packed_sequence(lstm_out,batch_first=True)
        hidden = self.dropout_op(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        avg_pool=self.avg_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        max_pool=self.max_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        
        #CNN Portion
        new_embs=embs.permute(0,2,1)        
        conved_0=torch.relu(self.conv_0(new_embs))
        conved_1=torch.relu(self.conv_1(new_embs))
        conved_2=torch.relu(self.conv_2(new_embs)) 
        max_pool1d=torch.nn.MaxPool1d(conved_0.shape[2])
        pooled_0=max_pool1d(conved_0).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_1.shape[2])
        pooled_1=max_pool1d(conved_1).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_2.shape[2])
        pooled_2=max_pool1d(conved_2).squeeze(2)
        cat_cnn = self.dropout_op(torch.cat([pooled_0,pooled_1,pooled_2],dim=1))
        
        ## Concatenate
        big_out=torch.cat([cat_cnn,hidden,max_pool],dim=1)
        preds=self.fc(big_out)

        loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))

        
        return preds.view(-1),loss
        
    

In [47]:
def accuracy_binomial(preds,actual, device="cpu", cutoff=0.5):
    preds=torch.sigmoid(preds)
    zeros=torch.zeros(len(preds)).to(device)
    ones = torch.ones(len(preds)).to(device)

    preds=torch.where(preds>cutoff,ones,zeros)
    correct=torch.round(preds).long()==actual
    return correct.float().sum()/len(correct)

In [48]:
def accuracy_binomial(preds, y, device="cpu", cutoff=0.5):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    y=y.float()
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [49]:
device="cpu"

## Load pretrained model weights

In [50]:
pretrained_lm_weights=pickle.load(open(f'{DATAPATH}/inter/varybpttpretrained_lm_weights','rb'))

### Check if model forward works

In [51]:
n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout_o,dropout,n_out

(1157, 400, 400, 2, True, 52, 'cpu', 0.2, 0.5, 0.5, 1)

In [52]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=dropout_e,dropout=dropout,\
                 dropout_o=dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)


initializing glove with (1157, 400)


In [53]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model_sentiment):,} trainable parameters')

The model has 7,357,801 trainable parameters


In [54]:
model_sentiment

sentiment_classifier(
  (dropout_enc): Dropout(p=0.2)
  (encoder): Embedding(1157, 400, padding_idx=1)
  (lstm): LSTM(400, 400, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout_op): Dropout(p=0.5)
  (avg_pool1d): AdaptiveAvgPool1d(output_size=1)
  (max_pool1d): AdaptiveMaxPool1d(output_size=1)
  (conv_0): Conv1d(400, 100, kernel_size=(3,), stride=(1,))
  (conv_1): Conv1d(400, 100, kernel_size=(4,), stride=(1,))
  (conv_2): Conv1d(400, 100, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=1900, out_features=1, bias=True)
  (criterion): BCEWithLogitsLoss()
)

In [55]:
model_sentiment.forward(xb,yb,xlen)

(tensor([-0.2957, -0.0987, -0.2387, -0.1701, -0.3535, -0.1752,  0.1103, -0.5423,
         -0.2065, -0.2461, -0.3308, -0.1905, -0.1277, -0.1963, -0.1232, -0.1079,
         -0.5029, -0.0280,  0.1083, -0.3913, -0.2970,  0.0062, -0.1091, -0.3339,
         -0.3287, -0.6138, -0.0052, -0.4315, -0.4681, -0.2631, -0.1125, -0.4644,
         -0.2301, -0.2871, -0.0772, -0.1500, -0.2684, -0.4205, -0.2378, -0.0919,
         -0.3249, -0.2560, -0.0753, -0.3428, -0.1779, -0.4347, -0.2086,  0.0415,
         -0.0501, -0.2903, -0.3605, -0.1835], grad_fn=<ViewBackward>),
 tensor(0.7255, grad_fn=<BinaryCrossEntropyWithLogitsBackward>))

In [56]:
preds,loss=model_sentiment.forward(xb,yb,xlen)

In [57]:
preds.view(-1)

tensor([-0.0866, -0.3287, -0.2888, -0.1230, -0.0411, -0.1927, -0.0738,  0.0723,
        -0.1654, -0.3878, -0.3026, -0.2871, -0.0866, -0.1831, -0.2745, -0.0347,
         0.1026, -0.0293, -0.2005, -0.4144, -0.4897, -0.0941, -0.0682, -0.0501,
        -0.3391, -0.1323, -0.3858, -0.3179, -0.4038, -0.1516, -0.0666, -0.3111,
        -0.2353, -0.5496, -0.1467, -0.3105, -0.1890, -0.0189, -0.5746, -0.1266,
        -0.2349, -0.7070, -0.0850, -0.2076, -0.1468, -0.1222, -0.1902, -0.2978,
        -0.1731, -0.5811, -0.7390, -0.1237], grad_fn=<ViewBackward>)

In [58]:
yb

tensor([0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
        0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 0, 1, 0])

In [59]:
preds.size()

torch.Size([52])

In [60]:
yb.size()

torch.Size([52])

In [61]:
accuracy_binomial(preds.to(device),yb.to(device))

tensor(0.4615)

In [62]:
from sklearn.metrics import roc_auc_score

In [63]:
roc_auc_score(yb,preds)

0.4075757575757576

In [64]:
pretrained_lm_weights

array([[-0.077831,  0.302613, -0.350404,  0.357022, ...,  0.363273, -0.073379, -0.035527,  0.140866],
       [-0.072569, -0.084968, -0.044134, -0.093804, ..., -0.063597, -0.056705,  0.049921, -0.134325],
       [-0.031278, -0.01621 ,  0.084576, -0.053553, ..., -0.016662, -0.014197,  0.135493,  0.191577],
       [ 0.1578  , -0.057542,  0.06689 , -0.211143, ...,  0.030297,  0.12647 , -0.128329,  0.193179],
       ...,
       [ 0.079816,  0.051887, -0.097742,  0.093509, ...,  0.016576,  0.067949, -0.03363 ,  0.154612],
       [-0.047566, -0.244655,  0.215301, -0.207936, ..., -0.226427, -0.100595, -0.104421,  0.190134],
       [-0.030737, -0.07049 ,  0.030162, -0.012834, ..., -0.012499,  0.031222,  0.029341,  0.153292],
       [-0.014552, -0.053913,  0.125428,  0.029424, ..., -0.137652, -0.211496,  0.034593, -0.120307]], dtype=float32)

## Learner

In [65]:
# Weight Decay Schedule
tempstart=1e-7
for i in range (6):
    print (i, tempstart)
    tempstart=tempstart*6

0 1e-07
1 6e-07
2 3.6e-06
3 2.16e-05
4 0.0001296
5 0.0007776


In [66]:
tempstart=5e-3
for i in range (6):
    print (i, tempstart)
    tempstart=tempstart*(0.7**i)

0 0.005
1 0.005
2 0.0034999999999999996
3 0.0017149999999999995
4 0.0005882449999999997
5 0.0001412376244999999


In [67]:
torch.randint(1,5,(1,))

tensor([1])

In [68]:
lrs=[1e-2,5e-3,1e-4,5e-4]

In [69]:
torch.randint(0,4,(1,))

tensor([0])

In [70]:
lrs[torch.randint(0,4,(1,))]

0.01

In [71]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None,\
                 cycle_mult=0,lr_decay=0.7,wd_mult=6):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
        self.cycle_mult,self.lr_decay=cycle_mult,lr_decay
        self.wd_mult=wd_mult
        for param_group in self.optimizer.param_groups:
            self.start_lr=param_group['lr']
            self.start_wd=param_group['weight_decay']
        self.wd=self.start_wd
        self.lr=self.start_lr
        self.n_epoch=0
        self.lrs=[1e-2,5e-3,1e-4,5e-4]
        self.preds,self.preds_valid,self.trainY,self.actual=[],[],[],[]
        
    def fit (self,Xb,Yb,Xlen,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb,Xlen)
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1),self.device)
            acc=acc.item()
            
            if mode_train:
                self.trainY.append(Yb.view(-1))
                self.preds.append(preds.data)
            else:
                self.actual.append(Yb.view(-1))
                self.preds_valid.append(preds.data)
            
            del preds
        
        if mode_train:
            if 1==0:
                lr =self.lrs[torch.randint(0,4,(1,))]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_val)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.init_hidden()
        for Xb,Yb,Xlen in iterator:
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            Xlen=Xlen.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,Xlen,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            
            k=k+1
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/len(iterator)
        epoch_acc=epoch_acc/len(iterator)
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

     
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        
        if self.cycle_mult > 0:
            reset_cycle=self.cycle_mult
        
        for epoch in range(n_epochs):                

            
            loss,acc=self.run_epoch(dltrain,True)
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Learning rate {self.lr} Weight Decay {self.wd} Train Loss:{loss} Train Accuracy:{acc} Valid Loss:{lossv} Valid Accuracy:{accv}')
        
            if self.cycle_mult:
                if self.n_epoch==reset_cycle:
                    self.lr=self.start_lr
                    #self.wd=self.start_wd
                    reset_cycle=self.n_epoch+reset_cycle
                else:
                    self.lr*=(self.lr_decay**self.n_epoch)  
                    if self.n_epoch>1:
                        self.wd*=self.wd_mult
            self.n_epoch+=1
                
                
            for param_group in self.optimizer.param_groups:
                param_group['lr']=self.lr
                #param_group['weight_decay']=self.wd

In [72]:
device="cuda:1"

In [73]:
n_layers

2

In [74]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout,\
                 dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)
model_sentiment=model_sentiment.to(device)

initializing glove with (1157, 400)


In [75]:
optimizer=torch.optim.Adam(model_sentiment.parameters(),lr=5e-3,betas=(0.9,0.999), weight_decay=wd)
#optimizer=torch.optim.SGD(model_sentiment.parameters(),lr=1e-2,momentum=0.9, weight_decay=wd)
metric_fn=accuracy_binomial

In [76]:
len(dltrain),len(dlvalid)

(30, 4)

In [77]:
df_valid['label'].value_counts()

0    111
1     84
Name: label, dtype: int64

In [78]:
learner=Learner(model_sentiment,optimizer,accuracy_binomial,device,bptt,100,0.25,cycle_mult=6)

In [79]:
model_sentiment.freeze_embedding()

In [80]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(False, False)

In [81]:
model_sentiment.dropout_e,model_sentiment.dropout,model_sentiment.dropout_o, learner.model.dropout_e,learner.model.dropout,learner.model.dropout_o

(0.2, 0.5, 0.5, 0.2, 0.5, 0.5)

In [82]:
learner.run_epochs(dltrain,dlvalid,5)

Epoch:0 Learning rate 0.005 Weight Decay 1e-07 Train Loss:0.3461472916106383 Train Accuracy:0.851125098268191 Valid Loss:0.10332920961081982 Valid Accuracy:0.9695513099431992
Epoch:1 Learning rate 0.005 Weight Decay 1e-07 Train Loss:0.20154877329866092 Train Accuracy:0.9312925557295482 Valid Loss:0.08145591989159584 Valid Accuracy:0.9839743822813034
Epoch:2 Learning rate 0.0034999999999999996 Weight Decay 1e-07 Train Loss:0.11509649381041527 Train Accuracy:0.955730019013087 Valid Loss:0.0485295420512557 Valid Accuracy:0.9807692617177963
Epoch:3 Learning rate 0.0017149999999999995 Weight Decay 6e-07 Train Loss:0.08867404207897683 Train Accuracy:0.9672684729099273 Valid Loss:0.03443378582596779 Valid Accuracy:0.988782063126564
Epoch:4 Learning rate 0.0005882449999999997 Weight Decay 3.6e-06 Train Loss:0.04421157588561376 Train Accuracy:0.9858974536259969 Valid Loss:0.01951175695285201 Valid Accuracy:0.9935897439718246


In [83]:
torch.save(model_sentiment.state_dict(),f'{DATAPATH}/inter/combo_ytsm')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/combo9_0.ytsm')
torch.save (model_sentiment,f'{DATAPATH}/inter/combo9_ytsm')
torch.save (optimizer,f'{DATAPATH}/inter/combo9_ytsm')
torch.save (learner,f'{DATAPATH}/inter/combo9_ytsm')

/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type sentiment_classifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [84]:
model_sentiment.unfreeze_embedding()

In [85]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(True, True)

In [87]:
from itertools import chain

In [91]:
preds=list(chain.from_iterable(learner.preds))[-df_train.shape[0]:]
preds_valid=list(chain.from_iterable(learner.preds_valid))[-df_valid.shape[0]:]
trainY=list(chain.from_iterable(learner.trainY))[-df_train.shape[0]:]
actual=list(chain.from_iterable(learner.actual))[-df_valid.shape[0]:]
preds=[x.item() for x in preds]
preds_valid=[x.item() for x in preds_valid]
trainY=[x.item() for x in trainY]
actual=[x.item() for x in actual]

In [92]:
from sklearn.metrics import roc_auc_score

In [96]:
roc_auc_score(trainY,preds),roc_auc_score(actual,preds_valid)

(0.9982127887097093, 1.0)

In [108]:
from sklearn.metrics import precision_score, recall_score

In [105]:
from scipy.special import expit

In [107]:
f1_score(trainY,np.round(expit(preds))), f1_score(actual,np.round(expit(preds_valid)))

(0.9859514687100894, 0.9940828402366864)

In [110]:
precision_score(trainY,np.round(expit(preds))), precision_score(actual,np.round(expit(preds_valid)))

(0.9922879177377892, 0.9882352941176471)

In [109]:
recall_score(trainY,np.round(expit(preds))), recall_score(actual,np.round(expit(preds_valid)))

(0.9796954314720813, 1.0)

In [99]:
#learner.run_epochs(dltrain,dlvalid,2)

Epoch:0 Learning rate 0.0001412376244999999 Weight Decay 2.16e-05 Train Loss:0.03245377823865662 Train Accuracy:0.9909864087899526 Valid Loss:0.008715230389498174 Valid Accuracy:1.0
Epoch:1 Learning rate 2.3737807549714975e-05 Weight Decay 0.0001296 Train Loss:0.030327929106230538 Train Accuracy:0.9897436082363129 Valid Loss:0.008683438412845135 Valid Accuracy:1.0


In [100]:
torch.save(model_sentiment.state_dict(),f'{DATAPATH}/inter/combo_ytsm_nofreeze')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/combo9_0.ytsm_nofreeze')
torch.save (model_sentiment,f'{DATAPATH}/inter/combo9_ytsm_nofreeze')
torch.save (optimizer,f'{DATAPATH}/inter/combo9_ytsm_nofreeze')
torch.save (learner,f'{DATAPATH}/inter/combo9_ytsm_nofreeze')